In [13]:
!pip install feast[aws] s3fs sklearn pandas

     |████████████████████████████████| 59 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 20.5 MB/s 
     |████████████████████████████████| 8.6 MB 21.7 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 58.4 MB/s 
     |████████████████████████████████| 271 kB 60.2 MB/s 
  Created wheel for aiobotocore: filename=aiobotocore-2.2.0-py3-none-any.whl size=57108 sha256=77202b1a2c36eb6b8dd988a495a57f913269061c322172291ce4ee9fdc4f48ea
  Stored in directory: /root/.cache/pip/wheels/5f/c1/90/d81f357ee479228ab98b7558b6a6af12234e5ee9c29ab5c16f
Successfully built aiobotocore
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.42
    Uninstalling botocore-1.24.42:
      Successfully uninstalled botocore-1.24.42
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.21.42 req

In [1]:
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn.cluster import KMeans

##Read the data and filter out data that belongs to country other than UK
retail_data = pd.read_csv('s3://feast-demo-mar-2022/data/OnlineRetail.csv', encoding= 'unicode_escape')
retail_data['InvoiceDate'] = pd.to_datetime(retail_data['InvoiceDate'], errors = 'coerce')
uk_data = retail_data.query("Country=='United Kingdom'").reset_index(drop=True)


In [2]:
## Create 3months and 6 months data frames
t1 = pd.Timestamp("2011-06-01 00:00:00.054000")
t2 = pd.Timestamp("2011-03-01 00:00:00.054000")
t3 = pd.Timestamp("2011-12-01 00:00:00.054000")
uk_data_3m = uk_data[(uk_data.InvoiceDate < t1) & (uk_data.InvoiceDate >= t2)].reset_index(drop=True)
uk_data_6m = uk_data[(uk_data.InvoiceDate >= t1) & (uk_data.InvoiceDate < t3)].reset_index(drop=True)


In [3]:
## Calculate RFM values.
uk_data_3m['revenue'] = uk_data_3m['UnitPrice'] * uk_data_3m['Quantity']
max_date = uk_data_3m['InvoiceDate'].max() + timedelta(days=1)
rfm_data = uk_data_3m.groupby(['CustomerID']).agg({
        'InvoiceDate': lambda x: (max_date - x.max()).days,
        'InvoiceNo': 'count',
        'revenue': 'sum'})
rfm_data.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'revenue': 'MonetaryValue'}, inplace=True)


In [4]:
## Calculate RFM groups of customers 
r_grp = pd.qcut(rfm_data['Recency'], q=4, labels=range(3,-1,-1))
f_grp = pd.qcut(rfm_data['Frequency'], q=4, labels=range(0,4))
m_grp = pd.qcut(rfm_data['MonetaryValue'], q=4, labels=range(0,4))
rfm_data = rfm_data.assign(R=r_grp.values).assign(F=f_grp.values).assign(M=m_grp.values)
rfm_data['R'] = rfm_data['R'].astype(int)
rfm_data['F'] = rfm_data['F'].astype(int)
rfm_data['M'] = rfm_data['M'].astype(int)
rfm_data['RFMScore'] = rfm_data['R'] + rfm_data['F'] + rfm_data['M']

In [5]:
# segment customers.
rfm_data['Segment'] = 'Low-Value'
rfm_data.loc[rfm_data['RFMScore']>4,'Segment'] = 'Mid-Value' 
rfm_data.loc[rfm_data['RFMScore']>6,'Segment'] = 'High-Value' 
rfm_data = rfm_data.reset_index()

In [6]:
# Calculate revenue using the six month dataframe.
uk_data_6m['revenue'] = uk_data_6m['UnitPrice'] * uk_data_6m['Quantity']
revenue_6m = uk_data_6m.groupby(['CustomerID']).agg({
        'revenue': 'sum'})
revenue_6m.rename(columns={'revenue': 'Revenue_6m'}, inplace=True)
revenue_6m = revenue_6m.reset_index()

In [7]:
# Merge the 6m revenue data frame with RFM data.
merged_data = pd.merge(rfm_data, revenue_6m, how="left")
merged_data.fillna(0)

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Segment,Revenue_6m
0,12747.0,7,35,1082.09,3,2,3,8,High-Value,1666.11
1,12748.0,1,582,4336.73,3,3,3,9,High-Value,18679.01
2,12749.0,8,54,782.10,3,3,3,9,High-Value,2323.04
3,12821.0,23,6,92.72,2,0,0,2,Low-Value,0.00
4,12823.0,63,1,459.00,0,0,2,2,Low-Value,765.00
...,...,...,...,...,...,...,...,...,...,...
1835,18272.0,21,59,966.74,2,3,3,8,High-Value,1730.16
1836,18273.0,66,1,51.00,0,0,0,0,Low-Value,102.00
1837,18280.0,86,10,180.60,0,0,0,0,Low-Value,0.00
1838,18283.0,9,100,217.15,3,3,1,7,High-Value,1351.83


In [8]:
# Create LTV cluster groups
merged_data = merged_data[merged_data['Revenue_6m']<merged_data['Revenue_6m'].quantile(0.99)]
kmeans = KMeans(n_clusters=3)
kmeans.fit(merged_data[['Revenue_6m']])
merged_data['LTVCluster'] = kmeans.predict(merged_data[['Revenue_6m']])

In [10]:
feature_data = pd.get_dummies(merged_data)
feature_data.head(5)

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Revenue_6m,LTVCluster,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value
0,12747.0,7,35,1082.09,3,2,3,8,1666.11,0,1,0,0
1,12748.0,1,582,4336.73,3,3,3,9,18679.01,1,1,0,0
2,12749.0,8,54,782.10,3,3,3,9,2323.04,0,1,0,0
4,12823.0,63,1,459.00,0,0,2,2,765.00,0,0,1,0
7,12836.0,28,62,814.71,1,3,3,7,951.46,0,1,0,0


In [11]:
feature_data.columns = ['customerid', 'recency', 'frequency', 'monetaryvalue', 'r', 'f', 'm',
       'rfmscore', 'revenue6m', 'ltvcluster', 'segmenthighvalue',
       'segmentlowValue', 'segmentmidvalue']

In [15]:
feature_data.head(5)

,customerid,recency,frequency,monetaryvalue,r,f,m,rfmscore,revenue6m,ltvcluster,segmenthighvalue,segmentlowValue,segmentmidvalue,event_timestamp,created_timestamp
0,12747.0,7,35,1082.09,3,2,3,8,1666.11,0,1,0,0,2022-04-16 15:05:34.380484,2022-04-16 15:05:34.384103
1,12748.0,1,582,4336.73,3,3,3,9,18679.01,1,1,0,0,2022-04-16 15:05:34.380484,2022-04-16 15:05:34.384103
2,12749.0,8,54,782.10,3,3,3,9,2323.04,0,1,0,0,2022-04-16 15:05:34.380484,2022-04-16 15:05:34.384103
4,12823.0,63,1,459.00,0,0,2,2,765.00,0,0,1,0,2022-04-16 15:05:34.380484,2022-04-16 15:05:34.384103
7,12836.0,28,62,814.71,1,3,3,7,951.46,0,1,0,0,2022-04-16 15:05:34.380484,2022-04-16 15:05:34.384103


In [ ]:
# import os
# os.environ["AWS_ACCESS_KEY_ID"] = "<aws_key>"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "<aws_secret>"
# os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

In [14]:
from datetime import datetime
file_name = f"rfm_features-{datetime.now()}.parquet" 
feature_data["event_timestamp"] = datetime.now()
feature_data["created_timestamp"] = datetime.now()
feature_data['customerid'] = feature_data['customerid'].astype(str)

In [ ]:

s3_url = f's3://feast-demo-mar-2022/customer-rfm-features/{file_name}'
feature_data.to_parquet(s3_url)